### MACS 30113/30123 Lab Session: Working with EMR Clusters/Spark
*Week 6*

**Adam Wu, Wonje Yun**


### Create S3 Bucket

First create S3 bucket to store our files.

In [6]:
import boto3

In [7]:
# Initialize boto3 handler
s3 = boto3.resource('s3')

# Create a new bucket to store your files
BUCKETNAME = 'adam-example-bucket'
s3.create_bucket(Bucket=BUCKETNAME)

# This is what we will use to interface with the specific bucket
bucket = s3.Bucket( BUCKETNAME )

In [8]:
# Upload your .py file to S3

FILENAME = 'mystuff/myfile.py'
with open('mystuff/myfile.py', 'rb') as myfile:
    bucket.put_object(Key=FILENAME, Body=myfile)

### Launching EMR Cluster

Next launch EMR Cluster in Terminal/bash.

In [5]:
%%bash 

aws emr create-cluster \
    --name "Spark Cluster" \
    --release-label "emr-6.2.0" \
    --applications Name=Hadoop Name=Hive Name=JupyterEnterpriseGateway Name=JupyterHub Name=Livy Name=Pig Name=Spark Name=Tez \
    --instance-type m5.xlarge \
    --instance-count 3 \
    --use-default-roles \
    --region us-east-1 \
    --ec2-attributes '{"KeyName": "vockey"}' \
    --configurations '[{"Classification": "jupyter-s3-conf", "Properties": {"s3.persistence.enabled": "true", "s3.persistence.bucket": "adam-example-bucket"}}]'


{
    "ClusterId": "j-2GGSSO7FCKI2X",
    "ClusterArn": "arn:aws:elasticmapreduce:us-east-1:348752177325:cluster/j-2GGSSO7FCKI2X"
}


When creating a new cluster, make sure to adjust the security settings to allow for `ssh` access. See `emr_cheatsheet.md` in Week 7 course materials.

#### Method 1: `ssh` Directly

The first way to work with EMR is to directly `ssh` into it, then work with it just like we did for `EC2` (see previous lab on EC2).

Connecting to it:
```
$ ssh -i vockey.pem hadoop@EMR-PUBLIC-ADDRESS
```

Uploading a folder called `mystuff` locally -> EMR:
```
$ scp -i "vockey.pem" -r mystuff hadoop@EMR-PUBLIC-ADDRESS:/home/hadoop
```

Downloading a folder called `mystuff` from EMR -> locally:
```
$ scp -i "vockey.pem" -r hadoop@EMR-PUBLIC-ADDRESS:/home/hadoop/mystuff .
```
---

After uploading your files in there, you can then run Spark jobs with
``` 
[EMR] spark-submit mystuff/myfile.py
```
Alternatively if your files are saved on `S3`, then
```
[EMR] spark-submit s3://adam-example-bucket/mystuff/myfile.py adam-example-bucket
```

#### Method 2: Interactive Sessions

You can also launch a Jupyter server directly on EMR and work with it interactively.
```
$ ssh -i "vockey.pem" -NL 9443:localhost:9443 hadoop@EMR-PUBLIC-ADDRESS
```
This forwards the remote connection to your `https://localhost:9443`, and you can log in with username `jovyan`, password `jupyter`. 

#### Method 3: Running Spark on Midway

Using `sbatch`, refer to `in-class-activities/07_Spark/7M_Spark_EDA_ML/midway` on Week 7 course materials. 

You can also work with Spark interactive on Midway with `sinteractive`:
```bash
$ sinteractive --ntasks=<NUMBER-OF-TASKS> --account=macs30123
```
This would take you into the computation node, where you can start `python3` or `ipython3` directly in the terminal. For Spark, you need to run `pyspark`.
```bash
$ pyspark
```
This would start the spark shell and pyspark, where you can work like `python3` in the terminal.
To exit from spark shell run:
```bash
$ exit()
```
To exit from sinteractive run:
```bash
$ exit()
```
